<a href="https://colab.research.google.com/github/fancagi/timetabling_upb/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import pulp as plp
import json

# read data

In [ ]:
horas_curso=json.loads(open('data/horas_curso.json').read())
idoneidad=json.loads(open('data/idoneidad.json').read())
horas_reglamento=json.loads(open('data/horas_reglamento.json').read())
labor_instruccional=json.loads(open('data/labor_instruccional.json').read())
grupo_requerimiento=json.loads(open('data/grupo_requerimiento.json').read())
ubicacion_semestral=json.loads(open('data/ubicacion_semestral.json').read())


In [ ]:
L={str(i):[str(grupo_requerimiento[i]['codigo'])+'-'+str(j) for j in range(grupo_requerimiento[i]['n_grupos'])] for i in grupo_requerimiento.keys()}   
curso_base={str(i)+"-"+str(j):str(i)  for i in grupo_requerimiento.keys() for j in range(grupo_requerimiento[i]['n_grupos']) }



In [ ]:
grupos=[]
for l in L.values():
    for ele in l:
        grupos.append(ele)

# Modelo

In [ ]:
# Conjuntos

I=set(idoneidad.keys()) # Profesores
J=set(grupos) # Grupos
K=set(grupo_requerimiento.keys()) # Cursos
# Conjunto auxiliar que sirve para enumerar la cantidad de cursos que un docente tiene asignados
C=set(range(1,7)) # Posibles valores para el número de clases en un semestre asignadas a un docente

In [ ]:

# combinaciones posibles de profesores ygrupo
tuplas=[(i,j) for i in I for j in J if i in idoneidad.keys() and curso_base[j] in idoneidad[i]['idoneidad'] ]
# subindices usados para denotar la carga asignada a un profesor
doc_carga=[(i,c) for i in I for c in C]
# tuplas profesor y curso
prof_curso=[(i,j) for i in I for j in idoneidad[i]['idoneidad'].keys()]

# Modelo

## Declaración de variables

In [ ]:
import pulp as plp

# Create the 'prob' variable to contain the problem data
prob = plp.LpProblem("Modelo William", plp.LpMaximize)

# Create problem variables from dictionary
# Docente i es asignado a grupo j (binaria)
x=plp.LpVariable.dicts("x",tuplas,0,1,plp.LpBinary)

# Docente i es asignado a c cursos diferentes
N=plp.LpVariable.dicts("N",doc_carga,0,1,plp.LpBinary)

# Un profesor es asignado a un curso
z=plp.LpVariable.dicts("z",prof_curso,0,1,plp.LpBinary)



/Users/user/opt/anaconda3/envs/apricot-env/lib/python3.9/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


## Funcion objetivo 1 
 maximizar la suma de la idoneidad de los profesores asignados a los diferentes grupos



In [ ]:
# e=(i,j) representa la tupla
prob+=plp.lpSum([x[(i,j)]*idoneidad[i]['idoneidad'][curso_base[j]] for (i,j) in tuplas])


### Restricciones

In [ ]:
# restriccion 1
cursos_excluir=['8833-0022','8833-0083','8833-0101','5568-0011','5562-0013']
# Cada curso tiene un docente asignado
for j in J:
    
    #
    
    if curso_base[j] not in cursos_excluir:
        prob+=plp.lpSum([x[i,j] for i in I if (i,j) in tuplas])==1,"asignacion curso %s"%j

        prob.solve(solver=plp.PULP_CBC_CMD(msg=0))
        if prob.status==-1:
            print("El problema es infactible")
            print(plp.lpSum([x[i,j] for i in I if (i,j) in tuplas])==1,"asignacion curso %s"%j)
            break


In [ ]:
# Contar cuantos cursos diferentes da un profesor
for i in I:
    prob+=plp.lpSum([z[(i,k)] for k in K if (i,k) in prof_curso])==plp.lpSum([l*N[i,l] for l in C])
# el profesor tiene una cantidad de cursos asignada
for i in I:
    prob+=plp.lpSum([N[i,l] for l in C])==1

# el profesor no puede dar mas cursos de los que tiene asignados
for i in I:
    for k in K:
        if (i,k) in prof_curso:
            plp.lpSum([x[(i,j)] for j in L[k] if (i,j) in tuplas])<=z[(i,k)]*len(L[k])

In [ ]:
prob.solve()

1

In [ ]:
dict_curso_prof

In [ ]:
for (i,j) in tuplas:
    if x[(i,j)].value()>0.5:
        try:
            print(i,j,curso_base[j],labor_instruccional[i]['docente'],horas_curso[curso_base[j]]["ASIGNATURA"])
        except:
            print(i,j,curso_base[j])

234541 2027-0059-0 2027-0059 Marcelo_antonioni_Aguilar_Montero Fundamentos_dibujo_CAD
234541 2027-0059-1 2027-0059 Marcelo_antonioni_Aguilar_Montero Fundamentos_dibujo_CAD
274691 8821-0002-0 8821-0002 Beatriz_Elena_Hernandez_Arias OCI-III_Htas_Toma_Desiciones
274691 8830-0012-0 8830-0012 Beatriz_Elena_Hernandez_Arias Sistemas_Gestión
274691 5566-0009-0 5566-0009 Beatriz_Elena_Hernandez_Arias Contexto_Profesional
274691 8830-0005-0 8830-0005 Beatriz_Elena_Hernandez_Arias Organización_Trabajo
331121 8833-0021-0 8833-0021 Edwin_Fernando_Restrepo_Salazar Metodología_Investigación
331121 9937-0013-0 9937-0013 Edwin_Fernando_Restrepo_Salazar Diseño_Experimental
331121 6069-0154-0 6069-0154 Edwin_Fernando_Restrepo_Salazar OCI_II_Propspectiva_Estrategica
277461 8830-0011-0 8830-0011 Mariela_Galindo_Barbosa Logística_Industrial
277461 6611-0088-0 6611-0088 Mariela_Galindo_Barbosa Ingeniería_Económica
277461 9937-0072-0 9937-0072 Mariela_Galindo_Barbosa Algebra_Lineal
277461 6069-0078-0 6069-007